In [1]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

The data is streamed and saved locally.
The saved data had a number of redundant files within collected subfolders of data 
such as files with.crc extension and files titled "_SUCCESS" or "._SUCCESS".
Only files titled "part-00000" and without crc extension are needed for the classification problem. 
Hence, only these files are retained. 
A total of 444 observations were obtained for review data.The collected data has much higher number of positive ratings 
relative to negative ratings. 
The retained files are then converted to a pandas dataframe and then stored locally in csv format. 

In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
from pyspark.streaming import StreamingContext

In [5]:
ssc = StreamingContext(sc,1)

In [6]:
lines = ssc.socketTextStream("seppe.net", 7778)

In [7]:
# Change the path below to match yours
lines.saveAsTextFiles("file:///C:/Users/RimJhim/Desktop/spark_instructions/")

In [8]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

In [31]:
# Don't run this cell unless you want to stop
ssc_t.stop()

----- Stopping... this may take a few seconds -----


In [9]:
import pandas as pd
os.chdir("C:/................../spark_instructions")
rootPath="/..................../spark_instructions/"
dirpath="/................../spark_instructions"
import json
d=None

for root,dirs,files in os.walk(rootPath):
    for file in os.scandir(root):
        #if file.is_file(): ##don't need it;still get correct result without it
            #print(file)
        #if str(file).find('part-0000')==1
            if (str(file).find('part')>1) and (str(file).find('crc')==-1):
               df2=pd.DataFrame(pd.read_json(file,lines=True))
               d=pd.concat([d,df2])
               print(d.shape)
              

(1, 4)
(2, 4)
(3, 4)
(4, 4)
(5, 4)
(6, 4)
(7, 4)
(8, 4)
(9, 4)
(10, 4)
(11, 4)
(12, 4)
(13, 4)
(14, 4)
(15, 4)
(16, 4)
(17, 4)
(18, 4)
(19, 4)
(20, 4)
(21, 4)
(22, 4)
(23, 4)
(24, 4)
(25, 4)
(26, 4)
(27, 4)
(28, 4)
(29, 4)
(30, 4)
(31, 4)
(32, 4)
(33, 4)
(34, 4)
(35, 4)
(36, 4)
(37, 4)
(38, 4)
(39, 4)
(40, 4)
(41, 4)
(42, 4)
(43, 4)
(44, 4)
(45, 4)
(46, 4)
(47, 4)
(48, 4)
(49, 4)
(50, 4)
(51, 4)
(52, 4)
(53, 4)
(54, 4)
(55, 4)
(56, 4)
(57, 4)
(58, 4)
(59, 4)
(60, 4)
(61, 4)
(62, 4)
(63, 4)
(64, 4)
(65, 4)
(66, 4)
(67, 4)
(68, 4)
(69, 4)
(70, 4)
(71, 4)
(72, 4)
(73, 4)
(74, 4)
(75, 4)
(76, 4)
(77, 4)
(78, 4)
(79, 4)
(80, 4)
(81, 4)
(82, 4)
(83, 4)
(84, 4)
(85, 4)
(86, 4)
(87, 4)
(88, 4)
(89, 4)
(90, 4)
(91, 4)
(92, 4)
(93, 4)
(94, 4)
(95, 4)
(96, 4)
(97, 4)
(98, 4)
(99, 4)
(100, 4)
(101, 4)
(102, 4)
(103, 4)
(104, 4)
(105, 4)
(106, 4)
(107, 4)
(108, 4)
(109, 4)
(110, 4)
(111, 4)
(112, 4)
(113, 4)
(114, 4)
(115, 4)
(116, 4)
(117, 4)
(118, 4)
(119, 4)
(120, 4)
(121, 4)
(122, 4)
(123, 4)
(

In [10]:
d.head()

,review_id,app_id,review_text,label
0,138198607,1277920,although this game still needs some work it ha...,1
0,138198396,1277920,It still is not playable. It has been two wee...,0
0,138235603,1277920,"If they keep at it the way they have so far, t...",1
0,138236287,1669000,Age of Wonders 3 on steroids. \n,1
0,138236059,1669000,Just... one more turn okay,1


In [116]:
d.head()

,review_id,app_id,review_text,label
0,138198607,1277920,although this game still needs some work it ha...,1
0,138198396,1277920,It still is not playable. It has been two wee...,0


In [11]:
d.shape

(444, 4)

In [30]:
d=d[d['review_text'].map(lambda x: x.isascii())] 
d.to_csv('d.csv')

In [16]:
d.groupby(['label']).size()
##data is not well-balanced-need more data 

label
0     99
1    345
dtype: int64

In [28]:
d["review_text"]

0    although this game still needs some work it ha...
0    It still is not playable.  It has been two wee...
0    If they keep at it the way they have so far, t...
0                     Age of Wonders 3 on steroids. \n
0                           Just... one more turn okay
                           ...                        
0    draw peter griffin\nget money.\n\ndidn't get m...
0    Super fun and challenging game, plus we learne...
0    So much fun and so cute! The perfect game not ...
0    A very fun game to play with friends after you...
0                                      JOHNNY THA GOAT
Name: review_text, Length: 444, dtype: object

In [29]:
d.head()

,review_id,app_id,review_text,label
0,138198607,1277920,although this game still needs some work it ha...,1
0,138198396,1277920,It still is not playable. It has been two wee...,0
0,138235603,1277920,"If they keep at it the way they have so far, t...",1
0,138236287,1669000,Age of Wonders 3 on steroids. \n,1
0,138236059,1669000,Just... one more turn okay,1
